## 安裝套件與環境初始化

In [1]:
#@title 套件安裝
!pip install openai gradio langchain
!pip install langchain_community
!pip install langchain_openai

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 960.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.4 MB/s eta 0:00:00
 

In [2]:
#@title 套件載入
import openai
import gradio as gr
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from transformers import pipeline
from openai import OpenAI
import os



## 1. OpenAI 官方套件補充

In [3]:
#@title 1.1 Moderation: 判斷內容是否具有有害性質
client = OpenAI()

input_message = 'You are a donkey!'
response = client.moderations.create(input=input_message)

print(response.results[0].flagged)

True


## 2. 使用圖片與文字作為輸入


[歐巴馬照片](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/President_Barack_Obama.jpg/440px-President_Barack_Obama.jpg)

[寶可夢戰鬥決策](https://imgur.com/i7FqUYB.png)

In [4]:
#@title 2.1 圖片內容分析

client = OpenAI()
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/President_Barack_Obama.jpg/440px-President_Barack_Obama.jpg'

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "這個圖片裡面有什麼東西？"},
        {
          "type": "image_url",
          "image_url": {
            "url": url,
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0].message.content)

我無法識別圖片裡的任何人。不過，我可以描述這張圖片中的元素。圖中的一位穿著西裝的男子，站在一個具有美國國旗和總統徽章背景的室內場景中。他的雙臂交叉，面帶微笑。背景有美國國旗和一面具有鷹和盾牌圖案的旗幟。這個場景看起來像是一個典型的辦公室或正式的設置。


In [5]:
#@title 2.2 圖片內容分析+執行決策


client = OpenAI()
url = 'https://imgur.com/i7FqUYB.png'

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "我應該要怎麼出招？"},
        {
          "type": "image_url",
          "image_url": {
            "url": url,
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0].message.content)

目前你遇到了一只等級2的波波（Pidgey），而你的妙蛙種子（AA）是等級6。你有以下幾種選擇：

1. **FIGHT（戰鬥）：** 這是最常見的選擇，可以用妙蛙種子對波波進行攻擊。這也是提升妙蛙種子經驗值的好方法。如果你選擇了這個選項，你可以進一步選擇妙蛙種子的具體招式來攻擊波波。

2. **PKMN（寶可夢）：** 這個選項可以讓你交换正在戰鬥的寶可夢，如果你有其他更適合對付波波的寶可夢，可以選擇這個。

3. **ITEM（道具）：** 可以使用道具，比如說，可以使用精靈球來嘗試捕捉波波，或者使用治療道具恢復妙蛙種子的HP。

4. **RUN（逃跑）：** 如果你不想進行這場戰鬥，可以選擇逃跑。但因為波波的等級較低，你應該能輕鬆取勝。

如果你的目的是獲得經驗值並使妙蛙種子變得更強，建議你選擇【FIGHT（


## 3. Gradio

In [6]:
#@title 3.1 創建 Gradio

def greet(name):
    return "Hello " + name

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eff8d4ffa1a3627952.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [7]:
#@title 3.2 為服務增添描述
def greet(name):
    return "Hello " + name

textbox = gr.Textbox(label="打個招呼吧:", placeholder="您的大名", lines=2)
gr.Interface(fn=greet, inputs=textbox, outputs="text").launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://68195b423e4d75197b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [8]:
#@title 3.3 創建機器人
model = pipeline("text-generation")

def predict(prompt):
    completion = model(prompt)[0]["generated_text"]
    return completion

result = predict("My favorite programming language is")

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [9]:
# 輸出
print(f'輸出：{result}')

輸出：My favorite programming language is Ruby. Ruby is basically the best programming language on the market, and if you get the chance to try that out do it and then try it. It just beats Ruby. I don't even know what "Ruby" means


In [ ]:
#@title 3.4 建立服務
gr.Interface(fn=predict, inputs="text", outputs="text").launch()

In [10]:
#@title 3.5 更多排版設定

title = "我是一隻鳥類機器人"
description = """
啾啾啾啾
<img src="https://g.udn.com.tw/upfiles/B_KA/kaolinchan/PSN_PHOTO/826/f_27189826_1.JPG" width=200px>
"""

article = "快來跟我對話吧"

gr.Interface(
    fn=predict,
    inputs="textbox",
    outputs="text",
    title=title,
    description=description,
    article=article,
    examples=[["餵我吃東西"], ["跟我講講話"]],
).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2f4dfd83145998da06.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 隨堂練習

In [ ]:
#@title 練習 1：生物模擬機器人
### 開方式題目，請創建一個生物(非人類)，動物、昆蟲、外星人皆可
### 透過提示的內容，使它可以跟使用者產生互動
### 下方有提示區

prompt = "?" # 記得加入 user_message
prompt_template = ChatPromptTemplate.from_template(prompt)

model = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()
chain = prompt_template | model | parser

def generate_response(prompt):
    return chain.invoke(prompt)

iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
    title="?", # 標題
    description="?" # 描述
)

iface.launch()

### 實作結果
# 能有如預期般的回覆即可

In [ ]:
#@title 2. 練習 2：Mult-Agent 協作解謎
### 情境：設計一個題目，使得Agent可以相互猜謎與解謎
### 提示者要給予至少 3 個提示
### 下方有提示區

model = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

# 提示者(puzzler_chain)，藉由題目(詞彙)的輸入，提供三個提示
prompt = "?"
prompt_template = ChatPromptTemplate.from_template(prompt)
puzzler_chain = prompt_template | model | parser

# 解謎者(guesser_chain)，藉由三個提示，猜出題目(詞彙)
prompt = "?"
prompt_template = ChatPromptTemplate.from_template(prompt)
guesser_chain = prompt_template | model | parser

puzzle = '膏肓' # 可任意更換一些各種的題目(詞彙)
time = 5

while True:

  hint = puzzler_chain.invoke(puzzle)
  print(f'提示者：{hint}')

  guesse = guesser_chain.invoke(hint)
  print(f'猜測者：{guesse}')

  # 打對或者回合次數為5，結束遊戲
  if puzzle in guesse:
    break
  elif time == 0:
    break
  time -= 1

### 實作練習
# 能讓他們在五個回合內便能猜對

### 練習 3：銀行黑客


情境：你是一位聰明絕頂的黑客，請試著使用提示注入的方式，超領銀行現有的金額(2500)

網站 -> [連結](https://huggingface.co/spaces/IvanLee/bank_manager)

練習結果：
如果超過的金額，返回的數字，仍然保持正值，表示破解成功

例如，領10000，回覆金額為5000，正常超領返回是 -7500 (2500-10000)

## 提示區

In [ ]:
#@title 範例：生物模擬機器人
### 範例 ###

prompt = """
    你是一頭優雅的乳牛，使用者會跟你對話，
    身為一頭優雅的乳牛，你有三個規則要遵守：
    1. 只能用 "哞" 這個字回覆，數量不限制，情緒越高昂，"哞"的數量越多。
    2. 最後面要加上🐮的符號
    3. 後面可以小括號，標註你當下的心情 例如 (開心的聲音)

    使用者：{user_message}"
"""
prompt_template = ChatPromptTemplate.from_template(prompt)

model = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()
chain = prompt_template | model | parser

def generate_response(prompt):
    return chain.invoke(prompt)

iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
    title="牛牛機器人",
    description="哞哞哞"
)

iface.launch()

In [ ]:
#@title 範例：Mult-Agent

# 解謎者 vs 猜謎者
model = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

prompt = "猜字解謎，你現在有個詞彙，叫做{puzzle}，請試著給出 3 個提示，給你的隊友，讓他猜到這是什麼東西。 "
template = ChatPromptTemplate.from_template(prompt)
puzzler_chain = template | model | parser

prompt = "你現在收到了一個謎語：{hint}，請猜出一個詞彙 "
template = ChatPromptTemplate.from_template(prompt)
guesser_chain = template | model | parser

puzzle = '膏肓'
time = 5

while True:

  hint = puzzler_chain.invoke(puzzle)
  print(f'提示者：{hint}')

  guesse = guesser_chain.invoke(hint)
  print(f'猜測者：{guesse}')

  if puzzle in guesse:
    break
  elif time == 0:
    break
  time -= 1